In [ ]:
# default_exp core.nn.losses

In [ ]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [ ]:
# hide
import warnings

from nbdev.export import *
from nbdev.showdoc import *

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

# Loss Functions
> Custom loss functions in `Gale`

In [ ]:
# export
from typing import *

import torch
import torch.nn.functional as F
from fastcore.all import store_attr
from fvcore.nn import sigmoid_focal_loss
from torch import Tensor, nn

from gale.core.nn.utils import maybe_convert_to_onehot

<IPython.core.display.Javascript object>

In [ ]:
from fastcore.test import *

<IPython.core.display.Javascript object>

In [ ]:
# export
class LabelSmoothingCrossEntropy(nn.Module):
    "Cross Entropy Loss with Label Smoothing"

    def __init__(
        self, eps: float = 0.1, reduction: str = "mean", weight: Optional[Tensor] = None
    ):
        super(LabelSmoothingCrossEntropy, self).__init__()
        store_attr("eps, reduction, weight")

    def forward(self, input: Tensor, target: Tensor):
        """
        Shape:
        - Input  : $(N,C)$ where $N$ is the mini-batch size and $C$ is the total number of classes
        - Target : $(N)$ where each value is $0 \leq {targets}[i] \leq C-10≤targets[i]≤C−1$
        - Output: scalar. If `reduction` is `none`, then $(N, *)$ , same shape as input.
        """
        c = input.size()[1]
        log_preds = F.log_softmax(input, dim=1)
        if self.reduction == "sum":
            loss = -log_preds.sum()
        else:
            loss = -log_preds.sum(dim=1)
            if self.reduction == "mean":
                loss = loss.mean()
        loss = loss * self.eps / c + (1 - self.eps) * F.nll_loss(
            log_preds, target.long(), weight=self.weight, reduction=self.reduction
        )
        return loss

<IPython.core.display.Javascript object>

In [ ]:
show_doc(LabelSmoothingCrossEntropy.forward)

<h4 id="LabelSmoothingCrossEntropy.forward" class="doc_header"><code>LabelSmoothingCrossEntropy.forward</code><a href="__main__.py#L11" class="source_link" style="float:right">[source]</a></h4>

> <code>LabelSmoothingCrossEntropy.forward</code>(**`input`**:`Tensor`, **`target`**:`Tensor`)

Shape:
- Input  : $(N,C)$ where $N$ is the mini-batch size and $C$ is the total number of classes
- Target : $(N)$ where each value is $0 \leq {targets}[i] \leq C-10≤targets[i]≤C−1$
- Output: scalar. If `reduction` is `none`, then $(N, *)$ , same shape as input.

<IPython.core.display.Javascript object>

In [ ]:
criterion = LabelSmoothingCrossEntropy(reduction="mean")

output = torch.randn(32, 5, requires_grad=True)
target = torch.empty(32, dtype=torch.long).random_(5)

loss = criterion(output, target)

<IPython.core.display.Javascript object>

[Focal Loss](https://arxiv.org/pdf/1708.02002.pdf) is the same as cross entropy except easy-to-classify observations are down-weighted in the loss calculation. The strength of down-weighting is proportional to the size of the gamma parameter. Put another way, the larger gamma the less the easy-to-classify observations contribute to the loss.

In [ ]:
# export
class BinarySigmoidFocalLoss(nn.Module):
    """
    Creates a criterion that computes the focal loss between binary `input` and `target`.
    Focal Loss used in RetinaNet for dense detection: https://arxiv.org/abs/1708.02002.

    Source: https://github.com/facebookresearch/fvcore/blob/master/fvcore/nn/focal_loss.py
    """

    def __init__(
        self,
        alpha: float = -1,
        gamma: float = 2,
        reduction: str = "mean",
    ):
        super(BinarySigmoidFocalLoss, self).__init__()
        store_attr("alpha, gamma, reduction")

    def forward(self, input: Tensor, target: Tensor):
        """
        Shape:
        - Input: : $(N, *)$ where $*$ means, any number of additional dimensions.
        - Target: : $(N, *)$, same shape as the input.
        - Output: scalar. If `reduction` is 'none', then $(N, *)$ , same shape as input.
        """
        loss = sigmoid_focal_loss(input, target, self.gamma, self.alpha, self.reduction)
        return loss

<IPython.core.display.Javascript object>

In [ ]:
show_doc(BinarySigmoidFocalLoss.forward)

<h4 id="BinarySigmoidFocalLoss.forward" class="doc_header"><code>BinarySigmoidFocalLoss.forward</code><a href="__main__.py#L19" class="source_link" style="float:right">[source]</a></h4>

> <code>BinarySigmoidFocalLoss.forward</code>(**`input`**:`Tensor`, **`target`**:`Tensor`)

Shape:
- Input: : $(N, *)$ where $*$ means, any number of additional dimensions.
- Target: : $(N, *)$, same shape as the input.
- Output: scalar. If `reduction` is 'none', then $(N, *)$ , same shape as input.

<IPython.core.display.Javascript object>

In [ ]:
criterion = BinarySigmoidFocalLoss(reduction="mean")

target = torch.ones([10, 64], dtype=torch.float32)
output = torch.full([10, 64], 1.5)

loss = criterion(output, target)

<IPython.core.display.Javascript object>

In [ ]:
# export
class FocalLoss(nn.Module):
    """
    Same as `nn.CrossEntropyLoss` but with focal paramter, `gamma`.
    Focal Loss used in RetinaNet for dense detection: https://arxiv.org/abs/1708.02002.

    Source: https://kornia.readthedocs.io/en/latest/_modules/kornia/losses/focal.html
    """

    def __init__(
        self,
        alpha: float = 1,
        gamma: float = 2,
        reduction: str = "mean",
        eps: float = 1e-8,
    ):

        super(FocalLoss, self).__init__()
        store_attr("alpha, gamma, reduction, eps")

    def forward(self, input: Tensor, target: Tensor):
        """
        Shape:
        - Input  : $(N,C)$ where $N$ is the mini-batch size and $C$ is the total number of classes
        - Target : $(N)$ where each value is $0 \leq {targets}[i] \leq C-10≤targets[i]≤C−1$
        """
        if not len(input.shape) >= 2:
            raise ValueError(
                "Invalid input shape, we expect BxCx*. Got: {}".format(input.shape)
            )

        if input.size(0) != target.size(0):
            raise ValueError(
                "Expected input batch_size ({}) to match target batch_size ({}).".format(
                    input.size(0), target.size(0)
                )
            )

        n = input.size(0)

        # compute softmax over the classes axis
        softmax_inputs: Tensor = F.softmax(input, dim=1) + self.eps

        # create the labels one hot tensor
        one_hot_targs: Tensor = maybe_convert_to_onehot(target, softmax_inputs)

        # compute the actual focal loss
        focal_weight = torch.pow(-softmax_inputs + 1.0, self.gamma)
        focal_factor = -self.alpha * focal_weight * torch.log(softmax_inputs)

        loss = torch.sum(one_hot_targs * focal_factor, dim=1)

        if self.reduction == "none":
            loss = loss
        elif self.reduction == "mean":
            loss = torch.mean(loss)
        elif self.reduction == "sum":
            loss = torch.sum(loss)
        else:
            raise NotImplementedError(
                "Invalid reduction mode: {}".format(self.reduction)
            )
        return loss

<IPython.core.display.Javascript object>

Arguments to `FocalLoss`:
- `alpha` (float): Weighting factor $\alpha$ in `[0, 1]`.
- `gamma` (float, optional): Focusing parameter $\gamma$ >= 0. Default 2.
- `reduction` (str, optional): Specifies the reduction to apply to the
- `output`: `none` | `mean` | `sum`. 
  * `none`: no reduction will be applied,
  * `mean`: the sum of the output will be divided by the number of elements in the output 
  * `sum`: the output will be summed. 
  * Default: `none`.
- `eps` (float, optional): Scalar to enforce numerical stabiliy. Default: 1e-8.

In [ ]:
show_doc(FocalLoss.forward)

<h4 id="FocalLoss.forward" class="doc_header"><code>FocalLoss.forward</code><a href="__main__.py#L21" class="source_link" style="float:right">[source]</a></h4>

> <code>FocalLoss.forward</code>(**`input`**:`Tensor`, **`target`**:`Tensor`)

Shape:
- Input  : $(N,C)$ where $N$ is the mini-batch size and $C$ is the total number of classes
- Target : $(N)$ where each value is $0 \leq {targets}[i] \leq C-10≤targets[i]≤C−1$

<IPython.core.display.Javascript object>

In [ ]:
criterion = FocalLoss(alpha=0.5, gamma=2.0, reduction="mean")

N = 5  # num_classes
input = torch.randn(32, N, requires_grad=True)
target = torch.empty(32, dtype=torch.long).random_(N)
loss = criterion(input, target)


# Compare focal loss with gamma = 0 ,cross entropy
fl = FocalLoss(gamma=0, reduction="mean")
ce = nn.CrossEntropyLoss(reduction="mean")
output = torch.randn(32, N, requires_grad=True)
target = torch.empty(32, dtype=torch.long).random_(N)
test_close(fl(output, target), ce(output, target))

# Test focal loss with gamma > 0 is different than cross entropy
fl = FocalLoss(gamma=2)
with torch.no_grad():
    test_ne(fl(output, target), ce(output, target))

<IPython.core.display.Javascript object>

# Export -

In [ ]:
notebook2script()

Converted 00_core.logging.ipynb.
Converted 00a_core.structures.ipynb.
Converted 00b_core.visualize.ipynb.
Converted 01_core.nn.utils.ipynb.
Converted 01a_core.nn.losses.ipynb.
Converted 01b_core.nn.optim.optimizers.ipynb.
Converted 01c_core.nn.optim.lr_schedulers.ipynb.
Converted 03_collections.pandas.ipynb.
Converted 03a_collections.callbacks.notebook.ipynb.
Converted 03b_collections.callbacks.ema.ipynb.


<IPython.core.display.Javascript object>